In [87]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os



In [88]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

In [89]:
in_file="raw_data_14-May'25.csv"
out_file="all_flow_data_13th May'25.csv"

In [90]:

raw_data=pd.read_csv(in_file,low_memory=False,memory_map=True)
data=pd.read_csv("format.csv",memory_map=True)
table_data=pd.read_csv("table_data.csv",memory_map=True)    



In [91]:
raw_data.head(3)

,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,first_qc_pending_min_created,sub_stage1,category,sub_category
0,1.129348e+07,09301547-f722-4109-aeae-4aacd808b0a2,1108896768,156,LEAD_SUCCESSFULLY_CLOSED,2025-05-08 16:08:23.000,2025-05-08 16:29:49.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,bsBWjk33412643658929,2019-07-27 10:59:22.000,General Trade QR_SB_EDC,2025-05-08 16:27:19.000,2025-05-08 16:29:04.000,2025-05-08 16:29:04.000,2025-05-08 16:27:19.000,NaN,Personal Services,Cyber Cafe
1,1.441923e+09,396e0e0b-75eb-4838-a00b-eb5e4fa96923,1421212705,156,LEAD_SUCCESSFULLY_CLOSED,2025-05-08 12:20:52.000,2025-05-08 12:38:48.000,NaN,NaN,INDIVIDUAL,"[""Paytm QR""]","{""PANEL_SUCCESS"":1}",True,cOFEEd39854350439714,2025-05-08 12:38:17.000,General Trade QR_SB_EDC,2025-05-08 12:36:13.000,2025-05-08 12:38:02.000,2025-05-08 12:38:02.000,2025-05-08 12:36:13.000,NaN,Retail and Shopping,Kirana
2,1.942887e+09,f4e42f6c-2083-4602-9520-f3a45cd8763a,1809440960,156,LEAD_SUCCESSFULLY_CLOSED,2025-01-18 13:08:19.000,2025-05-07 12:01:07.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,xQePrv25469891187743,2025-05-07 12:00:23.000,General Trade QR_SB_EDC,2025-05-07 11:56:22.000,2025-05-07 12:00:05.000,2025-05-07 12:00:05.000,2025-05-07 11:56:22.000,NaN,Retail and Shopping,Meat Shop


In [92]:

raw_data.insert(5,'q',0)
len(raw_data)

267917

In [93]:
table_map = table_data.set_index('NODE ID')['Lead Status'].to_dict()
raw_data['q'] = raw_data['workflow_node_id'].map(table_map)



In [94]:
raw_data.loc[raw_data['sub_stage1']=='QC_AUTO_APPROVED','q']='Leads QC Auto Approved'

In [95]:
raw_data['q'].value_counts()

q
Merchant Live                                                       154362
Lead Created/Posted (Pending at Sales team for  Document Upload)     87673
Documents Rejected (Pending at Sales Team)                           14705
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         1708
Leads QC Auto Approved                                                   7
Leads Rejected in Risk Review                                            4
Name: count, dtype: int64

In [96]:
raw_data=raw_data.dropna(subset=['q'])

raw_data['q'].value_counts()

q
Merchant Live                                                       154362
Lead Created/Posted (Pending at Sales team for  Document Upload)     87673
Documents Rejected (Pending at Sales Team)                           14705
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         1708
Leads QC Auto Approved                                                   7
Leads Rejected in Risk Review                                            4
Name: count, dtype: int64

In [97]:
raw_data['remove_date']=raw_data['updated_at'].copy()

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])

raw_data['remove_date'] = raw_data['remove_date'].dt.strftime('%Y-%m-%d')

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])
len(raw_data)


258459

In [98]:
#print(pd.Timestamp.now().normalize()-timedelta(days=1))

In [99]:

current_timestamp = pd.Timestamp.now().normalize()

raw_data=raw_data[raw_data['remove_date']!=current_timestamp]

In [100]:
raw_data=raw_data.drop(columns=['remove_date'])



In [101]:
raw_data['solution_type_level_3'].unique()

array([nan, 'KYC_UPDATE', 'ACCOUNT_UPGRADE_LENDING', 'PROFILE_UPDATE',
       'ACCOUNT_UPGRADE', 'DIY'], dtype=object)

In [102]:
raw_data['solution_type_level_3'].value_counts()

solution_type_level_3
KYC_UPDATE                 9599
ACCOUNT_UPGRADE_LENDING    4652
PROFILE_UPDATE               90
ACCOUNT_UPGRADE              44
DIY                           5
Name: count, dtype: int64

In [103]:
raw_data=raw_data[raw_data['solution_type_level_3']!='YBL_LIMIT_ENHANCEMENT']

raw_data['solution_type_level_3'].unique()

array([nan, 'KYC_UPDATE', 'ACCOUNT_UPGRADE_LENDING', 'PROFILE_UPDATE',
       'ACCOUNT_UPGRADE', 'DIY'], dtype=object)

In [104]:
raw_data['solution_type_level_3']=raw_data['solution_type_level_3'].replace(['ACCOUNT_UPGRADE_LENDING','PROFILE_UPDATE'],'ACCOUNT_UPGRADE')

raw_data['solution_type_level_3'].unique()

array([nan, 'KYC_UPDATE', 'ACCOUNT_UPGRADE', 'DIY'], dtype=object)

In [105]:
raw_data.loc[raw_data['agent_team']=='3P FSE','solution_type_level_3']='3P FSE'

In [106]:
raw_data['solution_type_level_3'].value_counts()

solution_type_level_3
KYC_UPDATE         9287
ACCOUNT_UPGRADE    4786
3P FSE              338
DIY                   5
Name: count, dtype: int64

In [107]:
print(len(raw_data))
raw_data.head(3)

253730


,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,q,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,first_qc_pending_min_created,sub_stage1,category,sub_category
0,1.129348e+07,09301547-f722-4109-aeae-4aacd808b0a2,1108896768,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-08 16:08:23.000,2025-05-08 16:29:49.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,bsBWjk33412643658929,2019-07-27 10:59:22.000,General Trade QR_SB_EDC,2025-05-08 16:27:19.000,2025-05-08 16:29:04.000,2025-05-08 16:29:04.000,2025-05-08 16:27:19.000,NaN,Personal Services,Cyber Cafe
1,1.441923e+09,396e0e0b-75eb-4838-a00b-eb5e4fa96923,1421212705,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-08 12:20:52.000,2025-05-08 12:38:48.000,NaN,NaN,INDIVIDUAL,"[""Paytm QR""]","{""PANEL_SUCCESS"":1}",True,cOFEEd39854350439714,2025-05-08 12:38:17.000,General Trade QR_SB_EDC,2025-05-08 12:36:13.000,2025-05-08 12:38:02.000,2025-05-08 12:38:02.000,2025-05-08 12:36:13.000,NaN,Retail and Shopping,Kirana
2,1.942887e+09,f4e42f6c-2083-4602-9520-f3a45cd8763a,1809440960,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-01-18 13:08:19.000,2025-05-07 12:01:07.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,xQePrv25469891187743,2025-05-07 12:00:23.000,General Trade QR_SB_EDC,2025-05-07 11:56:22.000,2025-05-07 12:00:05.000,2025-05-07 12:00:05.000,2025-05-07 11:56:22.000,NaN,Retail and Shopping,Meat Shop


In [108]:
data[['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp',
       'Product_Context', 'Name_Match_Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp',
       'Stage movement count', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category', 'sub_category','first_qc_pending_min_created']] = raw_data[['cust_id', 'lead_id', 'entity_type', 'workflow_node_id', 'sub_stage',
                                                                                                    'q', 'created_at', 'updated_at', 'products_context', 'name_match_status', 'final_timestamp', 'panelqc_max_created', 'qc_pending_max_created', 'sbstage_count', 'fse_cust_id', 'agent_team', 'mid', 'mid_creation_date', 'solution_type_level_3', 'category', 'sub_category','first_qc_pending_min_created']]
       
len(data)

253730

In [109]:
data.head(3)


,Merchant Cust ID,Lead_ID,Entity_Type,Node id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,Name_Match_Status,10K/1rs MID Status,Final TimeStamp,QC Panel Successful TimeStamp,Last QC Pending Timestamp,TAT,Merchant QC TAT slab,Working TimeSlab,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,category,sub_category,first_qc_pending_min_created
0,1.129348e+07,09301547-f722-4109-aeae-4aacd808b0a2,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-08 16:08:23.000,2025-05-08 16:29:49.000,NaN,NaN,True,NaN,2025-05-08 16:29:04.000,2025-05-08 16:29:04.000,2025-05-08 16:27:19.000,NaN,NaN,NaN,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1108896768,General Trade QR_SB_EDC,bsBWjk33412643658929,2019-07-27 10:59:22.000,NaN,Personal Services,Cyber Cafe,2025-05-08 16:27:19.000
1,1.441923e+09,396e0e0b-75eb-4838-a00b-eb5e4fa96923,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-08 12:20:52.000,2025-05-08 12:38:48.000,NaN,"[""Paytm QR""]",True,NaN,2025-05-08 12:38:02.000,2025-05-08 12:38:02.000,2025-05-08 12:36:13.000,NaN,NaN,NaN,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1421212705,General Trade QR_SB_EDC,cOFEEd39854350439714,2025-05-08 12:38:17.000,NaN,Retail and Shopping,Kirana,2025-05-08 12:36:13.000
2,1.942887e+09,f4e42f6c-2083-4602-9520-f3a45cd8763a,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-01-18 13:08:19.000,2025-05-07 12:01:07.000,NaN,NaN,True,NaN,2025-05-07 12:00:05.000,2025-05-07 12:00:05.000,2025-05-07 11:56:22.000,NaN,NaN,NaN,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1809440960,General Trade QR_SB_EDC,xQePrv25469891187743,2025-05-07 12:00:23.000,NaN,Retail and Shopping,Meat Shop,2025-05-07 11:56:22.000


In [110]:
data['Updated Timestamp']=pd.to_datetime(data['Updated Timestamp'])
data['Created Timestamp']=pd.to_datetime(data['Created Timestamp'])
data['QC Panel Successful TimeStamp']=pd.to_datetime(data['QC Panel Successful TimeStamp'])
data['first_qc_pending_min_created']=pd.to_datetime(data['first_qc_pending_min_created'])


In [111]:
#current_date = datetime.now() - timedelta(days=2)

#current_timestamp = pd.Timestamp.now().normalize()-timedelta(days=1)


#print(current_timestamp)

#print(current_date)

In [112]:
user_date = '2024-10-15'  # Example: YYYY-MM-DD format

# Convert the combined string into a datetime object
custom_timestamp = datetime.strptime(user_date, '%Y-%m-%d')

print(custom_timestamp)
print(type(custom_timestamp))

current_day = custom_timestamp.day
current_month = custom_timestamp.month
current_year = custom_timestamp.year

print(current_day)
print(current_month)
print(current_year)


2024-10-15 00:00:00
<class 'datetime.datetime'>
15
10
2024


In [113]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[(data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Last 7 days'

2025-05-07 18:11:26.618976


In [114]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[(data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Yesterday'

In [115]:
data['Ageing']=data['Ageing'].fillna('')

In [116]:
data['Ageing'].value_counts()

Ageing
Last 7 days    117396
               115071
Yesterday       21263
Name: count, dtype: int64

In [117]:


data.loc[data['Name_Match_Status']==True, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Matched','10k MID created']

data.loc[data['Name_Match_Status']==False, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Not Matched','1Rs MID created']

In [118]:
data['Name_Match_Status']=data['Name_Match_Status'].fillna('')

In [119]:
data['Name_Match_Status'].value_counts()

Name_Match_Status
Name Matched        157011
                     55469
Name Not Matched     41250
Name: count, dtype: int64

In [120]:
data['Product_Context'].value_counts()

Product_Context
["Paytm QR"]     133864
["EDC"]           14253
["TAP_N_PAY"]       412
[]                    2
Name: count, dtype: int64

In [121]:
data.loc[data['Product_Context'].isin(['[]','["EDC"]','["TAP_N_PAY"]']),'Product_Context']='EDC'
data['Product_Context'].value_counts()

Product_Context
["Paytm QR"]    133864
EDC              14667
Name: count, dtype: int64

In [122]:
data.loc[~data['Product_Context'].isin(['EDC']),'Product_Context']='NON_EDC'
data['Product_Context'].value_counts()

Product_Context
NON_EDC    239063
EDC         14667
Name: count, dtype: int64

In [123]:
data['mid_created_date']=pd.to_datetime(data['mid_created_date'],errors='coerce')

In [124]:
data['mid_created_date'].head(10)

0   2019-07-27 10:59:22
1   2025-05-08 12:38:17
2   2025-05-07 12:00:23
3   2025-05-05 12:11:05
4   2025-05-07 13:43:36
5   2025-05-07 12:41:41
6   2025-05-03 11:17:21
7   2022-10-31 10:14:48
8   2025-05-02 15:58:23
9   2025-05-06 11:15:43
Name: mid_created_date, dtype: datetime64[ns]

In [125]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category',
       'sub_category', 'first_qc_pending_min_created'],
      dtype='object')

In [126]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day<current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='More Than 7 Days'

2025-05-07 18:11:27.134680


In [127]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='6-7 Days'

In [128]:
current_date = datetime.now() - timedelta(days=5)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='4-5 Days'

In [129]:

current_date = datetime.now() - timedelta(days=3)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='2-3 Days'

In [130]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='Less Than 1 Day'

In [131]:
data['Rejection Ageing'].value_counts()

Rejection Ageing
More Than 7 Days    6609
6-7 Days            2538
4-5 Days            2408
Less Than 1 Day     1455
2-3 Days            1402
Name: count, dtype: int64

In [132]:
data['Rejection Ageing'].value_counts()

Rejection Ageing
More Than 7 Days    6609
6-7 Days            2538
4-5 Days            2408
Less Than 1 Day     1455
2-3 Days            1402
Name: count, dtype: int64

In [133]:
data.loc[~data['Entity_Type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'Entity_Type']='NON_PROPRIETORSHIP'

In [134]:
data['Entity_Type'].value_counts()

Entity_Type
INDIVIDUAL            233090
PROPRIETORSHIP         19162
NON_PROPRIETORSHIP      1478
Name: count, dtype: int64

In [135]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category',
       'sub_category', 'first_qc_pending_min_created'],
      dtype='object')

In [136]:
data['Lead Status'].value_counts()

Lead Status
Merchant Live                                                       152378
Lead Created/Posted (Pending at Sales team for  Document Upload)     85575
Documents Rejected (Pending at Sales Team)                           14412
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         1354
Leads QC Auto Approved                                                   7
Leads Rejected in Risk Review                                            4
Name: count, dtype: int64

In [137]:
null_rejected_timeStamp=data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]


null_rejected_timeStamp=null_rejected_timeStamp[['Lead_ID']]

print(len(null_rejected_timeStamp))

null_rejected_timeStamp['id']="'" + null_rejected_timeStamp['Lead_ID'] + "'" + ","

null_rejected_timeStamp.head(3)

null_rejected_timeStamp.to_csv("rejected_null.csv", index=False)


2


In [138]:
#dghdsg

In [139]:
# raise Exception("Stop execution here")

print(len(data))

253730


In [140]:
rejected_null_value=pd.read_csv('rejected_null_value.csv')

rejected_null_value['q']=pd.to_datetime(rejected_null_value['q'],format='%d-%m-%Y %H:%M:%S')

print(rejected_null_value.columns)

Index(['lead_id', 'q'], dtype='object')


In [141]:
print(rejected_null_value.head(3))
print(len(data))

                                lead_id                   q
0  5d40175a-32d3-499d-af9e-9c3b16af3f42 2025-04-24 12:56:24
1  d6143059-caa9-4353-93ff-cde985e787a5 2025-02-17 16:56:22
253730


In [142]:

# Aggregate the right DataFrame by taking the first occurrence per lead_id
#rejected_agg = rejected_null_value.groupby('lead_id').first().reset_index()

# Merge the DataFrames on the single column
#data = data.merge(rejected_agg[['lead_id', 'Status']], left_on='Lead_ID', right_on='lead_id', how='left')

# Fill in values based on conditions
#data.loc[(data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()), 'Last QC Pending Timestamp'] = data['q']


In [143]:
data=data.merge(rejected_null_value,left_on='Lead_ID',right_on='lead_id',how='left')

data.loc[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()) ,'Last QC Pending Timestamp']=data['q']

data=data.drop(columns=['lead_id','q'])

print(data.columns)
print(len(data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]))
print(len(data))

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category',
       'sub_category', 'first_qc_pending_min_created'],
      dtype='object')
0
253730


In [144]:
data['Final TimeStamp']=pd.to_datetime(data['Final TimeStamp'])
data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'])

In [145]:
# type(data['Final TimeStamp'][0])

print(type(data['Last QC Pending Timestamp'][0]))
print(len(data))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
253730


In [146]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category',
       'sub_category', 'first_qc_pending_min_created'],
      dtype='object')

In [147]:
data['Last QC Pending Timestamp'].head(10)

0   2025-05-08 16:27:19
1   2025-05-08 12:36:13
2   2025-05-07 11:56:22
3   2025-05-05 12:08:41
4   2025-05-07 13:41:47
5   2025-05-07 12:36:46
6   2025-05-03 11:07:24
7   2025-05-05 18:41:05
8                   NaT
9   2025-05-06 11:11:02
Name: Last QC Pending Timestamp, dtype: datetime64[ns]

In [148]:


data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')

# Define the start and end times
start_time = time(8, 30, 0)
end_time = time(21, 30, 0)



# Apply the condition to classify as 'Intime' or 'Offtime' based on the time component of 'N'
data['Working TimeSlab'] = data['Last QC Pending Timestamp'].apply(lambda x: 'Offtime' if pd.isna(x) else ('Intime' if start_time <= x.time() <= end_time else 'Offtime'))


data['Working TimeSlab'].head(10)

0     Intime
1     Intime
2     Intime
3     Intime
4     Intime
5     Intime
6     Intime
7     Intime
8    Offtime
9     Intime
Name: Working TimeSlab, dtype: object

In [149]:
data['Working TimeSlab'].value_counts()

Working TimeSlab
Intime     165792
Offtime     87938
Name: count, dtype: int64

In [150]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category',
       'sub_category', 'first_qc_pending_min_created'],
      dtype='object')

In [151]:
out_time_start = pd.to_datetime("21:30").time()
out_time_end = pd.to_datetime("23:59:59").time()
morning_time_end = pd.to_datetime("08:30").time()

data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')


# Apply conditions to create column R
def process_timestamp(row):
    timestamp = row['Last QC Pending Timestamp']
    
    # Check if timestamp is NaT
    if pd.isna(timestamp):
        return np.nan
    
    time = timestamp.time()
    
    if row['Working TimeSlab'] == 'Offtime':
        if out_time_start <= time <= out_time_end:
            # Condition 1: Increment date by 1
            return timestamp + pd.DateOffset(days=1)
        elif time < morning_time_end:
            # Condition 2: Return the same timestamp
            return timestamp
    # Condition 3: Return blank for 'intime'
    return np.nan


data['New Off Time'] = data.apply(process_timestamp, axis=1)

In [152]:
# Function to create S column based on R
def generate_s(row):
    if pd.notna(row['New Off Time']):  # If R is not NaT or blank
        # Set time to 08:30 on the same date as R
        return row['New Off Time'].replace(hour=8, minute=30, second=0)
    else:
        # If R is NaT or blank, copy the value from Q
        return row['Last QC Pending Timestamp']
    

data['New Last QC Pending Time'] = data.apply(generate_s, axis=1)

In [153]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category',
       'sub_category', 'first_qc_pending_min_created'],
      dtype='object')

In [154]:
# Calculate the difference between L and S and format it as HH:mm:ss
def calculate_difference(row):
    if isinstance(row['New Last QC Pending Time'], pd.Timestamp) and isinstance(row['Final TimeStamp'], pd.Timestamp):
        diff = row['Final TimeStamp'] - row['New Last QC Pending Time']
        return diff  # Extract HH:mm:ss from Timedelta
    return np.nan  # Return NaN if either L or S is not a timestamp

# Apply function to create Difference column
data['TAT'] = data.apply(calculate_difference, axis=1)

In [155]:
data['TAT'].head(10)

0   0 days 00:01:45
1   0 days 00:01:49
2   0 days 00:03:43
3   0 days 00:01:57
4   0 days 00:01:25
5   0 days 00:04:35
6   0 days 00:09:42
7   0 days 02:05:11
8               NaT
9   0 days 00:04:13
Name: TAT, dtype: timedelta64[ns]

In [156]:
# Update TAT where it is negative to be the difference between L and N
def adjust_negative_tat(row):
    if pd.notna(row['TAT']) and row['TAT'] < pd.Timedelta(0):  # Check if TAT is negative
        if isinstance(row['Final TimeStamp'], pd.Timestamp) and isinstance(row['Last QC Pending Timestamp'], pd.Timestamp):
            return row['Final TimeStamp'] - row['Last QC Pending Timestamp']  # Calculate L - N if TAT is negative
    return row['TAT']  # Keep original TAT if not negative

# Apply function to adjust negative TAT values
data['TAT'] = data.apply(adjust_negative_tat, axis=1)

In [157]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

      Final TimeStamp Last QC Pending Timestamp             TAT
0 2025-05-08 16:29:04       2025-05-08 16:27:19 0 days 00:01:45
1 2025-05-08 12:38:02       2025-05-08 12:36:13 0 days 00:01:49
2 2025-05-07 12:00:05       2025-05-07 11:56:22 0 days 00:03:43
3 2025-05-05 12:10:38       2025-05-05 12:08:41 0 days 00:01:57
4 2025-05-07 13:43:12       2025-05-07 13:41:47 0 days 00:01:25
5 2025-05-07 12:41:21       2025-05-07 12:36:46 0 days 00:04:35
6 2025-05-03 11:17:06       2025-05-03 11:07:24 0 days 00:09:42
7 2025-05-05 20:46:16       2025-05-05 18:41:05 0 days 02:05:11
8 2025-05-02 15:59:34                       NaT             NaT
9 2025-05-06 11:15:15       2025-05-06 11:11:02 0 days 00:04:13
253730


In [158]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

0

In [159]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'leads QCed in more than 01 hr'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)



In [160]:
data['Merchant QC TAT slab'].value_counts()

Merchant QC TAT slab
Leads QCed in between 02 - 05 min       67762
Leads QCed in less than 02 min          58843
Leads QCed in between 05 - 10 min       22330
Leads QCed in between 15 - 30 min        9393
Leads QCed in between 10 - 15 min        7431
Leads QCed in between 30 min - 01 hr      635
leads QCed in more than 01 hr             396
Name: count, dtype: int64

In [161]:
len(data)

253730

In [162]:
len(data[data['solution_type_level_3']=='3P FSE'])

338

In [163]:
data['solution_type_level_3'].value_counts()

solution_type_level_3
KYC_UPDATE         9287
ACCOUNT_UPGRADE    4786
3P FSE              338
DIY                   5
Name: count, dtype: int64

In [164]:
data = data[~(data['Product_Context'] == 'EDC')]
len(data)

239063

In [165]:
data['Product_Context'].value_counts()

Product_Context
NON_EDC    239063
Name: count, dtype: int64

In [166]:
#data['solution_type_level_3'].value_counts()
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'category',
       'sub_category', 'first_qc_pending_min_created'],
      dtype='object')

In [167]:
# Convert date columns to datetime
data['first_qc_pending_min_created'] = pd.to_datetime(data['first_qc_pending_min_created'], errors='coerce')


In [168]:
# Calculate the time difference (TAT) and handle NaT cases
data['end_to_end_tat'] = data['Final TimeStamp'] - data['first_qc_pending_min_created']
data['end_to_end_tat'] = data['TAT'].apply(lambda x: '' if pd.isna(x) else str(x))

In [169]:

# Categorize the time difference
def categorize_time(tat):
    if tat == '':
        return ''  # Return blank for missing TAT
    # Convert string timedelta back to timedelta object
    td = pd.to_timedelta(tat)
    total_minutes = td.total_seconds() / 60  # Convert timedelta to minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'Leads QCed in more than 01 hr'

# Apply the function to the DataFrame based on the conditions
conditions = data['Lead Status'].isin(['Merchant Live', 'Documents Rejected (Pending at Sales Team)'])
data.loc[conditions, 'End to End TAT slab'] = data.loc[conditions, 'end_to_end_tat'].apply(categorize_time)

In [170]:
data['End to End TAT slab'].value_counts()

End to End TAT slab
Leads QCed in between 02 - 05 min       67077
Leads QCed in less than 02 min          58823
Leads QCed in between 05 - 10 min       20401
Leads QCed in between 15 - 30 min        5393
Leads QCed in between 10 - 15 min        4746
Leads QCed in between 30 min - 01 hr      400
Leads QCed in more than 01 hr             177
Name: count, dtype: int64

In [171]:
data.head(3)

,Merchant Cust ID,Lead_ID,Entity_Type,Node id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,Name_Match_Status,10K/1rs MID Status,Final TimeStamp,QC Panel Successful TimeStamp,Last QC Pending Timestamp,TAT,Merchant QC TAT slab,Working TimeSlab,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,category,sub_category,first_qc_pending_min_created,end_to_end_tat,End to End TAT slab
0,1.129348e+07,09301547-f722-4109-aeae-4aacd808b0a2,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-08 16:08:23,2025-05-08 16:29:49,Last 7 days,NON_EDC,Name Matched,10k MID created,2025-05-08 16:29:04,2025-05-08 16:29:04,2025-05-08 16:27:19,0 days 00:01:45,Leads QCed in less than 02 min,Intime,NaT,2025-05-08 16:27:19,"{""PANEL_SUCCESS"":1}",NaN,1108896768,General Trade QR_SB_EDC,bsBWjk33412643658929,2019-07-27 10:59:22,NaN,Personal Services,Cyber Cafe,2025-05-08 16:27:19,0 days 00:01:45,Leads QCed in less than 02 min
1,1.441923e+09,396e0e0b-75eb-4838-a00b-eb5e4fa96923,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-08 12:20:52,2025-05-08 12:38:48,Last 7 days,NON_EDC,Name Matched,10k MID created,2025-05-08 12:38:02,2025-05-08 12:38:02,2025-05-08 12:36:13,0 days 00:01:49,Leads QCed in less than 02 min,Intime,NaT,2025-05-08 12:36:13,"{""PANEL_SUCCESS"":1}",NaN,1421212705,General Trade QR_SB_EDC,cOFEEd39854350439714,2025-05-08 12:38:17,NaN,Retail and Shopping,Kirana,2025-05-08 12:36:13,0 days 00:01:49,Leads QCed in less than 02 min
2,1.942887e+09,f4e42f6c-2083-4602-9520-f3a45cd8763a,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-01-18 13:08:19,2025-05-07 12:01:07,Last 7 days,NON_EDC,Name Matched,10k MID created,2025-05-07 12:00:05,2025-05-07 12:00:05,2025-05-07 11:56:22,0 days 00:03:43,Leads QCed in between 02 - 05 min,Intime,NaT,2025-05-07 11:56:22,"{""PANEL_SUCCESS"":1}",NaN,1809440960,General Trade QR_SB_EDC,xQePrv25469891187743,2025-05-07 12:00:23,NaN,Retail and Shopping,Meat Shop,2025-05-07 11:56:22,0 days 00:03:43,Leads QCed in between 02 - 05 min


In [172]:
data.to_csv(out_file, index=False)

In [173]:
sas

NameError: name 'sas' is not defined

In [ ]:


# Convert to datetime
data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'])
data['Final TimeStamp'] = pd.to_datetime(data['Final TimeStamp'])

# Define active hours (08:30 AM - 09:30 PM)
active_start_time = pd.to_timedelta('08:30:00')
active_end_time = pd.to_timedelta('21:30:00')

# Function to calculate active duration for each row
def calculate_active_duration(row):
    start = row['Last QC Pending Timestamp']
    end = row['Final TimeStamp']
    
    # Check if either start or end is NaT
    if pd.isna(start) or pd.isna(end):
        return "00:00:00"  # Return 0 hours if either is NaT
    
    # Initialize total active duration
    total_active_duration = pd.Timedelta(0)
    
    # Iterate over each date from start to end
    for single_date in pd.date_range(start=start.date(), end=end.date()):
        active_start = pd.Timestamp(single_date) + active_start_time
        active_end = pd.Timestamp(single_date) + active_end_time
        
        # Determine the effective active period for this day
        effective_start = max(start, active_start)
        effective_end = min(end, active_end)

        # If there's a valid active period, add its duration
        if effective_start < effective_end:
            total_active_duration += effective_end - effective_start

    # Convert total active duration to hours
    total_hours = total_active_duration.total_seconds() / 3600
    
    # Format the hours to HH:MM:SS
    hours = int(total_hours)
    minutes = int((total_hours - hours) * 60)
    seconds = int(((total_hours - hours) * 60 - minutes) * 60)
    
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Apply the function to each row
data['TAT'] = data.apply(calculate_active_duration, axis=1)

# Display the result
print(data[['Last QC Pending Timestamp', 'Final TimeStamp', 'TAT']])
print(len(data))


In [ ]:
# data['TAT']=data['Final TimeStamp']-data['Last QC Pending Timestamp']
# data['TAT']=data['TAT'].apply(lambda x: str(x).split()[-1] if pd.notna(x) else 'NaT')
print(data.columns)
print(len(data))

In [ ]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

In [ ]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

In [ ]:
data["TAT"]=pd.to_timedelta(data['TAT'])



print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))

In [ ]:
type(data['TAT'][0])

In [ ]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 16:
        return 'Leads QCed in less than 15 min'
    elif 16 <= total_minutes < 36:
        return 'Leads QCed in between 15-30 min'
    elif 36 <= total_minutes < 66:
        return 'Leads QCed in between 30 min -1 hr'
    elif 66 <= total_minutes < 126:
        return 'Leads QCed in between 1 - 2 hr'
    elif 126 <= total_minutes < 186:
        return 'Leads QCed in between 2-3 hr'
    elif 186 <= total_minutes < 246:
        return 'Leads QCed in between 3-4 hr'
    else:
        return 'leads QCed in more than 4 hrs'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)


    



In [ ]:
print(data['Merchant QC TAT slab'].value_counts())
print(len(data))

In [ ]:
data['TAT'].head(10)

In [ ]:

# Function to convert timedelta to HH:MM:SS format with cumulative hours
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)  # Convert total seconds to hours
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'  # Format as HH:MM:SS

# Apply the function to convert timedelta to HH:MM:SS
data['TAT'] = data['TAT'].apply(format_timedelta)

In [ ]:
data.loc[(data['Lead Status']!='Merchant Live') & (data['Lead Status']!='Documents Rejected (Pending at Sales Team)'),'TAT']=''

data['TAT'].head(10)

In [ ]:
data.columns

In [ ]:
data=data.drop(columns=['Working TimeSlab','New Off Time','New Last QC Pending Time','Node id'])
data.columns

In [ ]:
col=data.pop('TAT')

t=data.columns.get_loc('Merchant QC TAT slab')

data.insert(t+1,'TAT',col)

data.columns

In [ ]:
data['solution_type_level_3'].unique()
len(data)

In [ ]:
data_acup_kyc=data[data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]
data_mco=data[~data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]

In [ ]:
#raise Exception("Stop execution here")

In [ ]:
with pd.ExcelWriter("all_flow_data_21st Oct'24.xlsx") as writer:
    # Write the original dataframe to the first sheet
    data.to_excel(writer, sheet_name='All Data', index=False)
    
    # Write data filtered by Category 'A' and 'C' to the second sheet
    data_acup_kyc.to_excel(writer, sheet_name=' KYC & ACC UPGD', index=False)
    
    # Write data filtered by Category 'B' and 'D' to the third sheet
    data_mco.to_excel(writer, sheet_name='MCO', index=False)

In [ ]:
data.to_csv("all_flow_data_19th Oct'24.csv", index=False)
